In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pybaseball import statcast
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
#df= statcast(start_dt='2022-03-20', end_dt='2022-10-31')

In [3]:
#df.columns

In [4]:
#dft = df[['player_name','p_throws','pitch_type','release_speed','release_pos_x','release_pos_z','events','description','release_spin_rate','release_extension','plate_x','plate_z']]

In [5]:
#creating a new column for the vertical and horizontal break in inches
#dft.assign(pfx_z=df['pfx_z']*12)
#dft.assign(pfx_x=df['pfx_x']*12)
#dft.insert(5, 'Vertical Break', df['pfx_z']*12)
#dft.insert(6, 'Horizontal Break', df['pfx_x']*12)

In [6]:
#dft.rename(columns={'player_name':'Player Name','p_throws':'Throws','pitch_type':'Pitch Type','release_speed':'Velocity','release_pos_x':'Side Release','release_pos_z':'Release Height','events':'Events','description':'Description','release_spin_rate':'Spin Rate','release_extension':'Extension','pfx_x':'Horizontal Break','pfx_z':'Vertical Break','plate_x':'Side Location','plate_z':'Height Location'}, inplace=True)
#dft = dft[['Player Name','Throws','Pitch Type','Velocity','Spin Rate','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]


In [7]:
df = pd.read_csv('2022.csv')

In [8]:
df

,Unnamed: 0,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,155,"Pressly, Ryan",R,SL,89.3,2826.0,5.76,3.00,6.01,-0.43,6.3
1,157,"Pressly, Ryan",R,CU,80.1,3270.0,-13.80,16.56,6.07,-0.51,5.8
2,162,"Pressly, Ryan",R,SL,89.4,2872.0,3.36,3.72,6.10,-0.30,5.9
3,166,"Pressly, Ryan",R,CU,82.1,3261.0,-11.04,16.08,5.95,-0.37,6.0
4,171,"Pressly, Ryan",R,FF,93.8,2603.0,19.80,-3.00,5.97,-0.20,6.3
...,...,...,...,...,...,...,...,...,...,...,...
766444,2139,"Kay, Anthony",L,SL,85.7,2678.0,-2.40,-6.36,5.97,2.01,5.7
766445,2316,"Kay, Anthony",L,CH,86.5,1844.0,12.12,11.52,5.82,2.04,5.8
766446,2560,"Kay, Anthony",L,FF,93.8,2389.0,16.32,6.36,6.14,1.95,5.7
766447,2747,"Kay, Anthony",L,FF,93.6,2386.0,14.64,6.48,6.01,1.94,5.7


In [9]:
#multply Horizontal Break by -1 to get the correct value
df['Horizontal Break'] = df['Horizontal Break'] * -1
#muliply Side Release by -1 to get the correct value
df['Side Release'] = df['Side Release'] * -1

In [10]:
#load other data
pf = pd.read_csv('2023 College TM.csv')

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1636387468.py:2: DtypeWarning: Columns (27,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  pf = pd.read_csv('2023 College TM.csv')


In [11]:
pf.head()

,PitchNo,Date,Time,PAofInning,PitchofPA,Pitcher,PitcherId,PitcherThrows,PitcherTeam,Batter,...,ThrowTrajectoryZc1,ThrowTrajectoryZc2,PitchReleaseConfidence,PitchLocationConfidence,PitchMovementConfidence,HitLaunchConfidence,HitLandingConfidence,CatcherThrowCatchConfidence,CatcherThrowReleaseConfidence,CatcherThrowLocationConfidence
0,1,2023-05-18,18:06:43.82,1.0,1.0,"Olson, Mason",1.000097e+09,Left,BYU_COU,"Schoen, Grant",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2023-05-18,18:06:48.82,1.0,2.0,"Olson, Mason",1.000097e+09,Left,BYU_COU,"Schoen, Grant",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2023-05-18,18:06:53.82,1.0,3.0,"Olson, Mason",1.000097e+09,Left,BYU_COU,"Schoen, Grant",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2023-05-18,18:06:58.82,2.0,1.0,"Olson, Mason",1.000097e+09,Left,BYU_COU,"Johnson, Ryan",...,NaN,NaN,High,High,High,High,Low,NaN,NaN,NaN
4,5,2023-05-18,18:07:21.38,2.0,2.0,"Olson, Mason",1.000097e+09,Left,BYU_COU,"Johnson, Ryan",...,NaN,NaN,High,High,High,High,Medium,NaN,NaN,NaN


In [12]:
#Create a new dataframe with only the columns we want for Analysis
pft = pf[['Pitcher','PitcherThrows','AutoPitchType','RelSpeed','SpinRate','Tilt','RelHeight','RelSide','Extension','InducedVertBreak', 'HorzBreak']]


In [13]:
#deletes all rows with NaN values in the dataframe
pft = pft.dropna()

In [14]:
#rename columns
pft.rename(columns={'Pitcher':'Player Name','PitcherThrows':'Throws','AutoPitchType':'Pitch Type','RelSpeed':'Velocity','SpinRate':'Spin Rate','Tilt':'Tilt','InducedVertBreak':'Vertical Break','HorzBreak':'Horizontal Break','RelHeight':'Release Height','RelSide':'Side Release','Extension':'Extension'}, inplace=True)
pft.head()

,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Tilt,Release Height,Side Release,Extension,Vertical Break,Horizontal Break
3,"Olson, Mason",Left,Four-Seam,90.04033,2279.692299,11:00,5.36156,-2.62682,5.62469,11.19394,-5.13146
4,"Olson, Mason",Left,Sinker,90.76079,2295.342252,10:30,5.40170,-2.56155,5.61961,9.58985,-9.13831
5,"Olson, Mason",Left,Changeup,82.55002,1929.765355,9:45,5.12269,-3.01832,5.95223,6.95748,-13.50709
6,"Olson, Mason",Left,Slider,79.78570,2625.965794,2:45,4.93499,-3.18512,5.51928,1.96354,10.14989
7,"Olson, Mason",Left,Curveball,77.74631,2492.914939,3:45,4.69918,-3.31391,5.32632,-2.28508,8.50032


In [15]:
#delete rows with NaN values
pft = pft.dropna()
pft.head()


,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Tilt,Release Height,Side Release,Extension,Vertical Break,Horizontal Break
3,"Olson, Mason",Left,Four-Seam,90.04033,2279.692299,11:00,5.36156,-2.62682,5.62469,11.19394,-5.13146
4,"Olson, Mason",Left,Sinker,90.76079,2295.342252,10:30,5.40170,-2.56155,5.61961,9.58985,-9.13831
5,"Olson, Mason",Left,Changeup,82.55002,1929.765355,9:45,5.12269,-3.01832,5.95223,6.95748,-13.50709
6,"Olson, Mason",Left,Slider,79.78570,2625.965794,2:45,4.93499,-3.18512,5.51928,1.96354,10.14989
7,"Olson, Mason",Left,Curveball,77.74631,2492.914939,3:45,4.69918,-3.31391,5.32632,-2.28508,8.50032


In [16]:
df.head()

,Unnamed: 0,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,155,"Pressly, Ryan",R,SL,89.3,2826.0,5.76,-3.00,6.01,0.43,6.3
1,157,"Pressly, Ryan",R,CU,80.1,3270.0,-13.80,-16.56,6.07,0.51,5.8
2,162,"Pressly, Ryan",R,SL,89.4,2872.0,3.36,-3.72,6.10,0.30,5.9
3,166,"Pressly, Ryan",R,CU,82.1,3261.0,-11.04,-16.08,5.95,0.37,6.0
4,171,"Pressly, Ryan",R,FF,93.8,2603.0,19.80,3.00,5.97,0.20,6.3


In [17]:
#rename throws values to match the other dataframe
df['Throws'] = df['Throws'].replace(['R'],'Right')
df['Throws'] = df['Throws'].replace(['L'],'Left')
df.head()

,Unnamed: 0,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,155,"Pressly, Ryan",Right,SL,89.3,2826.0,5.76,-3.00,6.01,0.43,6.3
1,157,"Pressly, Ryan",Right,CU,80.1,3270.0,-13.80,-16.56,6.07,0.51,5.8
2,162,"Pressly, Ryan",Right,SL,89.4,2872.0,3.36,-3.72,6.10,0.30,5.9
3,166,"Pressly, Ryan",Right,CU,82.1,3261.0,-11.04,-16.08,5.95,0.37,6.0
4,171,"Pressly, Ryan",Right,FF,93.8,2603.0,19.80,3.00,5.97,0.20,6.3


In [18]:
#delete Unnamed:0 column
df = df.drop(['Unnamed: 0'], axis=1)

In [19]:
df

,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,"Pressly, Ryan",Right,SL,89.3,2826.0,5.76,-3.00,6.01,0.43,6.3
1,"Pressly, Ryan",Right,CU,80.1,3270.0,-13.80,-16.56,6.07,0.51,5.8
2,"Pressly, Ryan",Right,SL,89.4,2872.0,3.36,-3.72,6.10,0.30,5.9
3,"Pressly, Ryan",Right,CU,82.1,3261.0,-11.04,-16.08,5.95,0.37,6.0
4,"Pressly, Ryan",Right,FF,93.8,2603.0,19.80,3.00,5.97,0.20,6.3
...,...,...,...,...,...,...,...,...,...,...
766444,"Kay, Anthony",Left,SL,85.7,2678.0,-2.40,6.36,5.97,-2.01,5.7
766445,"Kay, Anthony",Left,CH,86.5,1844.0,12.12,-11.52,5.82,-2.04,5.8
766446,"Kay, Anthony",Left,FF,93.8,2389.0,16.32,-6.36,6.14,-1.95,5.7
766447,"Kay, Anthony",Left,FF,93.6,2386.0,14.64,-6.48,6.01,-1.94,5.7


In [20]:
#in df, multiply horizontal break by -1 to match the other dataframe


In [21]:
pft

,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Tilt,Release Height,Side Release,Extension,Vertical Break,Horizontal Break
3,"Olson, Mason",Left,Four-Seam,90.04033,2279.692299,11:00,5.36156,-2.62682,5.62469,11.19394,-5.13146
4,"Olson, Mason",Left,Sinker,90.76079,2295.342252,10:30,5.40170,-2.56155,5.61961,9.58985,-9.13831
5,"Olson, Mason",Left,Changeup,82.55002,1929.765355,9:45,5.12269,-3.01832,5.95223,6.95748,-13.50709
6,"Olson, Mason",Left,Slider,79.78570,2625.965794,2:45,4.93499,-3.18512,5.51928,1.96354,10.14989
7,"Olson, Mason",Left,Curveball,77.74631,2492.914939,3:45,4.69918,-3.31391,5.32632,-2.28508,8.50032
...,...,...,...,...,...,...,...,...,...,...,...
1402683,"Simeone, William",Right,Cutter,85.45072,1738.353419,12:45,6.94192,-0.18536,7.06488,18.48647,6.28217
1402684,"Simeone, William",Right,Four-Seam,86.26443,1830.367099,12:30,7.11235,-0.23868,6.75426,20.30552,6.29489
1402685,"Simeone, William",Right,Changeup,86.84356,1873.433583,1:00,6.89892,-0.12414,7.07404,18.22166,10.62075
1402686,"Simeone, William",Right,Slider,80.63980,1832.299899,11:15,6.90584,-0.03302,6.80905,9.99453,-3.40559


In [22]:
#concatenate the two dataframes
df = pd.concat([df,pft], ignore_index=True)

In [23]:
df

,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Tilt
0,"Pressly, Ryan",Right,SL,89.30000,2826.000000,5.76000,-3.00000,6.01000,0.43000,6.30000,NaN
1,"Pressly, Ryan",Right,CU,80.10000,3270.000000,-13.80000,-16.56000,6.07000,0.51000,5.80000,NaN
2,"Pressly, Ryan",Right,SL,89.40000,2872.000000,3.36000,-3.72000,6.10000,0.30000,5.90000,NaN
3,"Pressly, Ryan",Right,CU,82.10000,3261.000000,-11.04000,-16.08000,5.95000,0.37000,6.00000,NaN
4,"Pressly, Ryan",Right,FF,93.80000,2603.000000,19.80000,3.00000,5.97000,0.20000,6.30000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2137305,"Simeone, William",Right,Cutter,85.45072,1738.353419,18.48647,6.28217,6.94192,-0.18536,7.06488,12:45
2137306,"Simeone, William",Right,Four-Seam,86.26443,1830.367099,20.30552,6.29489,7.11235,-0.23868,6.75426,12:30
2137307,"Simeone, William",Right,Changeup,86.84356,1873.433583,18.22166,10.62075,6.89892,-0.12414,7.07404,1:00
2137308,"Simeone, William",Right,Slider,80.63980,1832.299899,9.99453,-3.40559,6.90584,-0.03302,6.80905,11:15


In [24]:
#delete Tilt
df = df.drop(['Tilt'], axis=1)

In [25]:
#group by player name and pitch type without heirarchy include Throws column
df_avg_pitch = df.groupby(['Player Name','Pitch Type','Throws'], as_index=False).mean()

In [26]:
df_avg_pitch.head()

,Player Name,Pitch Type,Throws,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,"Camarda, Brad",Changeup,Right,77.085644,2043.477195,16.007712,13.758622,5.916989,1.375978,6.064864
1,"Camarda, Brad",Curveball,Right,75.551580,1785.533995,1.912192,-2.619799,5.918850,1.299285,5.641056
2,"Camarda, Brad",Four-Seam,Right,86.058372,2151.847504,20.857697,7.826656,5.968720,1.035166,6.131466
3,"Camarda, Brad",Slider,Right,79.134562,1819.714531,4.909138,-3.336860,5.906344,1.204252,5.886594
4,"Reynolds, Cole",Changeup,Left,80.202321,1433.025889,8.363340,-12.588893,5.506492,-2.795116,6.401376


In [27]:
##################

In [28]:
#rename pitch types
df['Pitch Type'] = df['Pitch Type'].replace(['FF'],'Four-Seam')
df['Pitch Type'] = df['Pitch Type'].replace(['SL'],'Slider')
df['Pitch Type'] = df['Pitch Type'].replace(['CH'],'Changeup')
df['Pitch Type'] = df['Pitch Type'].replace(['CU'],'Curveball')
df['Pitch Type'] = df['Pitch Type'].replace(['FT'],'Two-Seam Fastball')
df['Pitch Type'] = df['Pitch Type'].replace(['FC'],'Cutter')
df['Pitch Type'] = df['Pitch Type'].replace(['FS'],'Splitter')
df['Pitch Type'] = df['Pitch Type'].replace(['SI'],'Sinker')
df['Pitch Type'] = df['Pitch Type'].replace(['KC'],'Knuckle Curve')
df.head()


,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,"Pressly, Ryan",Right,Slider,89.3,2826.0,5.76,-3.00,6.01,0.43,6.3
1,"Pressly, Ryan",Right,Curveball,80.1,3270.0,-13.80,-16.56,6.07,0.51,5.8
2,"Pressly, Ryan",Right,Slider,89.4,2872.0,3.36,-3.72,6.10,0.30,5.9
3,"Pressly, Ryan",Right,Curveball,82.1,3261.0,-11.04,-16.08,5.95,0.37,6.0
4,"Pressly, Ryan",Right,Four-Seam,93.8,2603.0,19.80,3.00,5.97,0.20,6.3


In [29]:
#csv to desktop
df.to_csv('2022_combines.csv')

In [30]:
df_avg = pd.read_csv('2022_combines.csv')

In [31]:
#unique pitch types
df_avg['Pitch Type'].unique()

array(['Slider', 'Curveball', 'Four-Seam', 'Changeup', 'Sinker', 'Cutter',
       'Knuckle Curve', 'Splitter', 'ST', 'PO', 'SV', 'CS', 'FA', 'EP',
       nan, 'KN', 'SC', 'Other'], dtype=object)

In [32]:
#remove pitch types that are acronyms
df_avg = df_avg[df_avg['Pitch Type'] != 'ST']
df_avg = df_avg[df_avg['Pitch Type'] != 'PO']
df_avg = df_avg[df_avg['Pitch Type'] != 'SV']   
df_avg = df_avg[df_avg['Pitch Type'] != 'CS']
df_avg = df_avg[df_avg['Pitch Type'] != 'FA']
df_avg = df_avg[df_avg['Pitch Type'] != 'EP']
df_avg = df_avg[df_avg['Pitch Type'] != 'nan']
df_avg = df_avg[df_avg['Pitch Type'] != 'KN']
df_avg = df_avg[df_avg['Pitch Type'] != 'SC']
df_avg = df_avg[df_avg['Pitch Type'] != 'Other']

In [33]:
#remove pitch types that are acronyms in df
df = df[df['Pitch Type'] != 'ST']
df = df[df['Pitch Type'] != 'PO']
df = df[df['Pitch Type'] != 'SV']
df = df[df['Pitch Type'] != 'CS']
df = df[df['Pitch Type'] != 'FA']
df = df[df['Pitch Type'] != 'EP']
df = df[df['Pitch Type'] != 'nan']
df = df[df['Pitch Type'] != 'KN']
df = df[df['Pitch Type'] != 'SC'] 
df = df[df['Pitch Type'] != 'Other']

In [34]:
df['Pitch Type'].unique()

array(['Slider', 'Curveball', 'Four-Seam', 'Changeup', 'Sinker', 'Cutter',
       'Knuckle Curve', 'Splitter', nan], dtype=object)

In [35]:
df

,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,"Pressly, Ryan",Right,Slider,89.30000,2826.000000,5.76000,-3.00000,6.01000,0.43000,6.30000
1,"Pressly, Ryan",Right,Curveball,80.10000,3270.000000,-13.80000,-16.56000,6.07000,0.51000,5.80000
2,"Pressly, Ryan",Right,Slider,89.40000,2872.000000,3.36000,-3.72000,6.10000,0.30000,5.90000
3,"Pressly, Ryan",Right,Curveball,82.10000,3261.000000,-11.04000,-16.08000,5.95000,0.37000,6.00000
4,"Pressly, Ryan",Right,Four-Seam,93.80000,2603.000000,19.80000,3.00000,5.97000,0.20000,6.30000
...,...,...,...,...,...,...,...,...,...,...
2137305,"Simeone, William",Right,Cutter,85.45072,1738.353419,18.48647,6.28217,6.94192,-0.18536,7.06488
2137306,"Simeone, William",Right,Four-Seam,86.26443,1830.367099,20.30552,6.29489,7.11235,-0.23868,6.75426
2137307,"Simeone, William",Right,Changeup,86.84356,1873.433583,18.22166,10.62075,6.89892,-0.12414,7.07404
2137308,"Simeone, William",Right,Slider,80.63980,1832.299899,9.99453,-3.40559,6.90584,-0.03302,6.80905


In [36]:
#remove nan values
df = df.dropna()


In [37]:
df['Pitch Type'].unique()

array(['Slider', 'Curveball', 'Four-Seam', 'Changeup', 'Sinker', 'Cutter',
       'Knuckle Curve', 'Splitter'], dtype=object)

In [38]:
#seperate df_avg into different dataframes for each pitch type four seam sinket together
df_avg_fastball = df_avg[df_avg['Pitch Type'].isin(['Four-Seam','Two-Seam Fastball','Sinker'])]
df_avg_slider = df_avg[df_avg['Pitch Type'].isin(['Slider'])]
df_avg_changeup = df_avg[df_avg['Pitch Type'].isin(['Changeup'])]
df_avg_curveball = df_avg[df_avg['Pitch Type'].isin(['Curveball'])]

In [39]:
#sepereate df into different dataframes for each pitch type four seam sinker together
df_fastball = df[df['Pitch Type'].isin(['Four-Seam','Two-Seam Fastball','Sinker'])]
df_slider = df[df['Pitch Type'].isin(['Slider'])]
df_changeup = df[df['Pitch Type'].isin(['Changeup'])]
df_curveball = df[df['Pitch Type'].isin(['Curveball'])]


In [40]:
#check the value type of df_avg_fastball
df.dtypes

Player Name          object
Throws               object
Pitch Type           object
Velocity            float64
Spin Rate           float64
Vertical Break      float64
Horizontal Break    float64
Release Height      float64
Side Release        float64
Extension           float64
dtype: object

In [41]:
#export to csv
df_avg_fastball.to_csv('2022_fastball.csv')
df_avg_slider.to_csv('2022_slider.csv')
df_avg_changeup.to_csv('2022_changeup.csv')
df_avg_curveball.to_csv('2022_curveball.csv')


In [42]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans


In [43]:
data_fb = pd.read_csv('2022_fastball.csv')

In [44]:

#remove Cluster
#df_fastball = df_fastball.drop(['Cluster'], axis=1)

In [45]:
#delete Unnamed: 0 column and Unnamed: 0.1 columm
data_fb = data_fb.drop(['Unnamed: 0'], axis=1)
data_fb = data_fb.drop(['Unnamed: 0.1'], axis=1)

In [46]:
df

,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,"Pressly, Ryan",Right,Slider,89.30000,2826.000000,5.76000,-3.00000,6.01000,0.43000,6.30000
1,"Pressly, Ryan",Right,Curveball,80.10000,3270.000000,-13.80000,-16.56000,6.07000,0.51000,5.80000
2,"Pressly, Ryan",Right,Slider,89.40000,2872.000000,3.36000,-3.72000,6.10000,0.30000,5.90000
3,"Pressly, Ryan",Right,Curveball,82.10000,3261.000000,-11.04000,-16.08000,5.95000,0.37000,6.00000
4,"Pressly, Ryan",Right,Four-Seam,93.80000,2603.000000,19.80000,3.00000,5.97000,0.20000,6.30000
...,...,...,...,...,...,...,...,...,...,...
2137305,"Simeone, William",Right,Cutter,85.45072,1738.353419,18.48647,6.28217,6.94192,-0.18536,7.06488
2137306,"Simeone, William",Right,Four-Seam,86.26443,1830.367099,20.30552,6.29489,7.11235,-0.23868,6.75426
2137307,"Simeone, William",Right,Changeup,86.84356,1873.433583,18.22166,10.62075,6.89892,-0.12414,7.07404
2137308,"Simeone, William",Right,Slider,80.63980,1832.299899,9.99453,-3.40559,6.90584,-0.03302,6.80905


In [47]:

# Define which columns need to be normalized and which need to be one-hot encoded
num_cols = ['Velocity', 'Spin Rate', 'Vertical Break', 'Horizontal Break', 'Release Height', 'Side Release', 'Extension']
cat_cols = ['Player Name','Throws', 'Pitch Type']

In [48]:
#remove nan values
df = df.dropna()

In [49]:
df

,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,"Pressly, Ryan",Right,Slider,89.30000,2826.000000,5.76000,-3.00000,6.01000,0.43000,6.30000
1,"Pressly, Ryan",Right,Curveball,80.10000,3270.000000,-13.80000,-16.56000,6.07000,0.51000,5.80000
2,"Pressly, Ryan",Right,Slider,89.40000,2872.000000,3.36000,-3.72000,6.10000,0.30000,5.90000
3,"Pressly, Ryan",Right,Curveball,82.10000,3261.000000,-11.04000,-16.08000,5.95000,0.37000,6.00000
4,"Pressly, Ryan",Right,Four-Seam,93.80000,2603.000000,19.80000,3.00000,5.97000,0.20000,6.30000
...,...,...,...,...,...,...,...,...,...,...
2137305,"Simeone, William",Right,Cutter,85.45072,1738.353419,18.48647,6.28217,6.94192,-0.18536,7.06488
2137306,"Simeone, William",Right,Four-Seam,86.26443,1830.367099,20.30552,6.29489,7.11235,-0.23868,6.75426
2137307,"Simeone, William",Right,Changeup,86.84356,1873.433583,18.22166,10.62075,6.89892,-0.12414,7.07404
2137308,"Simeone, William",Right,Slider,80.63980,1832.299899,9.99453,-3.40559,6.90584,-0.03302,6.80905


In [50]:
#unique
df['Throws'].unique()

array(['Right', 'Left', 'Both', 'Undefined'], dtype=object)

In [51]:
#sepereate by Throws == Right and Throws == Left
df_right = df[df['Throws'].isin(['Right'])]
df_left = df[df['Throws'].isin(['Left'])]

In [52]:
#unique pitchj types
df_right['Pitch Type'].unique()

array(['Slider', 'Curveball', 'Four-Seam', 'Changeup', 'Sinker', 'Cutter',
       'Knuckle Curve', 'Splitter'], dtype=object)

In [53]:
#sepereate by pitch type
df_right_fb = df_right[df_right['Pitch Type'].isin(['Four-Seam','Two-Seam Fastball','Sinker'])]
df_right_slider = df_right[df_right['Pitch Type'].isin(['Slider'])]
df_right_changeup = df_right[df_right['Pitch Type'].isin(['Changeup'])]
df_right_curveball = df_right[df_right['Pitch Type'].isin(['Curveball','Knuckle Curve'])]

In [54]:
#sepereate by pitch type
df_left_fb = df_left[df_left['Pitch Type'].isin(['Four-Seam','Two-Seam Fastball','Sinker'])]
df_left_slider = df_left[df_left['Pitch Type'].isin(['Slider'])]
df_left_changeup = df_left[df_left['Pitch Type'].isin(['Changeup'])]
df_left_curveball = df_left[df_left['Pitch Type'].isin(['Curveball','Knuckle Curve'])]

In [55]:
#create a new column nexted to velocity called Velo_percentile
df_right_fb['Velo_percentile'] = df_right_fb['Velocity'].rank(pct=True)
df_right_slider['Velo_percentile'] = df_right_slider['Velocity'].rank(pct=True)
df_right_changeup['Velo_percentile'] = df_right_changeup['Velocity'].rank(pct=True)
df_right_curveball['Velo_percentile'] = df_right_curveball['Velocity'].rank(pct=True)

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3386129439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_fb['Velo_percentile'] = df_right_fb['Velocity'].rank(pct=True)
/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3386129439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_slider['Velo_percentile'] = df_right_slider['Velocity'].rank(pct=True)
/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3386129439.py:4: SettingWithCopyWarning

In [56]:
df_left_fb['Velo_percentile'] = df_left_fb['Velocity'].rank(pct=True)
df_left_slider['Velo_percentile'] = df_left_slider['Velocity'].rank(pct=True)
df_left_changeup['Velo_percentile'] = df_left_changeup['Velocity'].rank(pct=True)
df_left_curveball['Velo_percentile'] = df_left_curveball['Velocity'].rank(pct=True)

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1799258935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_fb['Velo_percentile'] = df_left_fb['Velocity'].rank(pct=True)
/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1799258935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_slider['Velo_percentile'] = df_left_slider['Velocity'].rank(pct=True)
/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1799258935.py:3: SettingWithCopyWarning: 
A

In [57]:
#give percentile to extension
df_right_fb['Extension_percentile'] = df_right_fb['Extension'].rank(pct=True)
df_right_slider['Extension_percentile'] = df_right_slider['Extension'].rank(pct=True)
df_right_changeup['Extension_percentile'] = df_right_changeup['Extension'].rank(pct=True)
df_right_curveball['Extension_percentile'] = df_right_curveball['Extension'].rank(pct=True)

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3678988664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_fb['Extension_percentile'] = df_right_fb['Extension'].rank(pct=True)
/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3678988664.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_slider['Extension_percentile'] = df_right_slider['Extension'].rank(pct=True)
/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3678988664.py:4: SettingWit

In [58]:
df_left_fb['Extension_percentile'] = df_left_fb['Extension'].rank(pct=True)
df_left_slider['Extension_percentile'] = df_left_slider['Extension'].rank(pct=True)
df_left_changeup['Extension_percentile'] = df_left_changeup['Extension'].rank(pct=True)
df_left_curveball['Extension_percentile'] = df_left_curveball['Extension'].rank(pct=True)


/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1402796105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_fb['Extension_percentile'] = df_left_fb['Extension'].rank(pct=True)
/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1402796105.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_slider['Extension_percentile'] = df_left_slider['Extension'].rank(pct=True)
/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1402796105.py:3: SettingWithCop

In [59]:
df_left_fb

,Player Name,Throws,Pitch Type,Velocity,Spin Rate,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Velo_percentile,Extension_percentile
20,"Hand, Brad",Left,Sinker,92.60000,2364.000000,10.56000,-17.40000,6.57000,-2.40000,5.60000,0.729143,0.192215
23,"Hand, Brad",Left,Sinker,92.70000,2351.000000,11.16000,-19.68000,6.56000,-2.22000,5.70000,0.739318,0.243816
25,"Hand, Brad",Left,Sinker,93.00000,2343.000000,13.44000,-17.16000,6.60000,-2.34000,5.70000,0.768668,0.243816
26,"Hand, Brad",Left,Sinker,93.30000,2353.000000,12.84000,-16.92000,6.57000,-2.50000,5.60000,0.796424,0.192215
27,"Hand, Brad",Left,Sinker,92.40000,2376.000000,15.12000,-15.84000,6.60000,-2.38000,5.60000,0.708490,0.192215
...,...,...,...,...,...,...,...,...,...,...,...,...
2137209,"Hughes, Nolan",Left,Four-Seam,91.42848,1987.233121,17.88170,-2.87138,6.88616,-1.39394,6.42043,0.600869,0.741819
2137210,"Hughes, Nolan",Left,Four-Seam,90.57230,2083.456929,20.17926,-6.60370,6.72292,-1.58033,6.29119,0.494189,0.623231
2137212,"Hughes, Nolan",Left,Four-Seam,90.51745,1989.319259,18.78092,-7.57166,6.82237,-1.50298,6.02284,0.489363,0.453042
2137213,"Hughes, Nolan",Left,Four-Seam,90.00482,2026.680500,19.24855,-3.02319,6.76926,-1.65346,6.43030,0.424775,0.744728


In [60]:
#take the mean and group by player name no multi index / hierarchy
df_right_fb = df_right_fb.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_right_slider = df_right_slider.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_right_changeup = df_right_changeup.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_right_curveball = df_right_curveball.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()

In [61]:
df_left_fb = df_left_fb.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_left_slider = df_left_slider.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_left_changeup = df_left_changeup.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()
df_left_curveball = df_left_curveball.groupby(['Player Name','Throws','Pitch Type'], as_index=False).mean()

In [62]:
#create a new column next to player name called player name ID and set it equal to index
df_right_fb['Player Name ID'] = df_right_fb.index
df_right_slider['Player Name ID'] = df_right_slider.index
df_right_changeup['Player Name ID'] = df_right_changeup.index
df_right_curveball['Player Name ID'] = df_right_curveball.index
df_left_fb['Player Name ID'] = df_left_fb.index
df_left_slider['Player Name ID'] = df_left_slider.index
df_left_changeup['Player Name ID'] = df_left_changeup.index
df_left_curveball['Player Name ID'] = df_left_curveball.index

In [63]:
df_right_fb_velo = df_right_fb[['Player Name','Player Name ID','Velo_percentile']]
df_right_slider_velo = df_right_slider[['Player Name','Player Name ID','Velo_percentile']]
df_right_changeup_velo = df_right_changeup[['Player Name','Player Name ID','Velo_percentile']]
df_right_curveball_velo = df_right_curveball[['Player Name','Player Name ID','Velo_percentile']]

In [64]:
df_left_fb_velo = df_left_fb[['Player Name','Player Name ID','Velo_percentile']]
df_left_slider_velo = df_left_slider[['Player Name','Player Name ID','Velo_percentile']]
df_left_changeup_velo = df_left_changeup[['Player Name','Player Name ID','Velo_percentile']]
df_left_curveball_velo = df_left_curveball[['Player Name','Player Name ID','Velo_percentile']]

In [65]:
df_right_fb_movement = df_right_fb[['Vertical Break','Horizontal Break','Player Name ID']]
df_right_slider_movement = df_right_slider[['Vertical Break','Horizontal Break','Player Name ID']]
df_right_changeup_movement = df_right_changeup[['Vertical Break','Horizontal Break','Player Name ID']] 
df_right_curveball_movement = df_right_curveball[['Vertical Break','Horizontal Break','Player Name ID']]


In [66]:
df_left_fb_movement = df_left_fb[['Vertical Break','Horizontal Break','Player Name ID']]
df_left_slider_movement = df_left_slider[['Vertical Break','Horizontal Break','Player Name ID']]
df_left_changeup_movement = df_left_changeup[['Vertical Break','Horizontal Break','Player Name ID']]
df_left_curveball_movement = df_left_curveball[['Vertical Break','Horizontal Break','Player Name ID']]

In [67]:
df_right_fb_release = df_right_fb[['Release Height','Side Release','Player Name ID']]
df_right_slider_release = df_right_slider[['Release Height','Side Release','Player Name ID']]
df_right_changeup_release = df_right_changeup[['Release Height','Side Release','Player Name ID']]
df_right_curveball_release = df_right_curveball[['Release Height','Side Release','Player Name ID']] 


In [68]:
df_left_fb_release = df_left_fb[['Release Height','Side Release','Player Name ID']]
df_left_slider_release = df_left_slider[['Release Height','Side Release','Player Name ID']]
df_left_changeup_release = df_left_changeup[['Release Height','Side Release','Player Name ID']]
df_left_curveball_release = df_left_curveball[['Release Height','Side Release','Player Name ID']]

In [69]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_fb_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_fb_movement['Cluster'] = movement_labels


In [70]:
df_right_fb_movement

In [71]:
#cluster summary
df_right_fb_movement.groupby('Cluster').mean()

In [72]:
#rename clusters based on pitch description
df_right_fb_movement['Cluster'] = df_right_fb_movement['Cluster'].replace({0:'Slight-Ride',1:'Slight Sink ',2:'Cut-Ride',3:'Sink',4:'Ride'})

In [73]:
#value counts by cluster
df_right_fb_movement['Cluster'].value_counts()

In [74]:
df_right_fb_movement.head(25)

In [75]:
#Change the name of cluster column to Pitch Movement
df_right_fb_movement = df_right_fb_movement.rename(columns={'Cluster':'Pitch Movement'})

In [76]:
#replace Player Name ID with Player Name from df_right_fb
df_right_fb_movement = df_right_fb_movement.replace({'Player Name ID':df_right_fb['Player Name']})

In [77]:
df_right_fb_movement

In [78]:
#export to csv
df_right_fb_movement.to_csv('2022_right_fb_movement.csv')

In [79]:
#import csv
df_right_fb_movement = pd.read_csv('2022_right_fb_movement.csv')

In [80]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data2 = scaler.fit_transform(df_right_fb_release)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data2)


# Add the labels to your dataframe
df_right_fb_release['Cluster'] = movement_labels

In [81]:
#summary of clusters
df_right_fb_release.groupby('Cluster').mean()

In [82]:
#count of clusters
df_right_fb_release['Cluster'].value_counts()

In [83]:
#rename clusters based on Release description
df_right_fb_release['Cluster'] = df_right_fb_release['Cluster'].replace({0:'3/4',1:'3/4',2:'OverTheTop',3:'Low 3/4 | Sidearm',4:'OverTheTop'})

In [84]:
df_right_fb_release

In [85]:
#Change the name of cluster column to Pitch Release
df_right_fb_release = df_right_fb_release.rename(columns={'Cluster':'Pitch Release'})

In [86]:
#replace Player Name ID with Player Name from df_right_fb
df_right_fb_release = df_right_fb_release.replace({'Player Name ID':df_right_fb['Player Name']})

In [87]:
#export to csv
df_right_fb_release.to_csv('2022_right_fb_release.csv')

In [88]:
#import csv
df_right_fb_release = pd.read_csv('2022_right_fb_release.csv')

In [89]:
#combine movement and release dataframes for right handed fastballs based on Player Name ID
df_right_fb_movement_release = pd.merge(df_right_fb_movement, df_right_fb_release, on='Player Name ID', how='outer')

In [90]:
df_right_fb_movement_release

,Unnamed: 0_x,Vertical Break,Horizontal Break,Player Name ID,Pitch Movement,Unnamed: 0_y,Release Height,Side Release,Pitch Release
0,0,20.857697,7.826656,"Camarda, Brad",Cut-Ride,0,5.968720,1.035166,OverTheTop
1,1,14.189920,11.144770,"26, Player",Slight-Ride,1,5.796032,1.503844,OverTheTop
2,1,14.189920,11.144770,"26, Player",Slight-Ride,2,5.671890,1.488883,3/4
3,2,11.069930,17.108043,"26, Player",Slight-Ride,1,5.796032,1.503844,OverTheTop
4,2,11.069930,17.108043,"26, Player",Slight-Ride,2,5.671890,1.488883,3/4
...,...,...,...,...,...,...,...,...,...
13941,7709,15.297171,8.149070,"palmer, blaine",Ride,7709,5.864502,2.232665,3/4
13942,7709,15.297171,8.149070,"palmer, blaine",Ride,7710,5.848382,2.374415,3/4
13943,7710,11.892075,11.506785,"palmer, blaine",Slight Sink,7709,5.864502,2.232665,3/4
13944,7710,11.892075,11.506785,"palmer, blaine",Slight Sink,7710,5.848382,2.374415,3/4


In [91]:
#df_right_fb_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_right_fb_velo['Cluster'] = pd.cut(df_right_fb_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])



/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3033308090.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_fb_velo['Cluster'] = pd.cut(df_right_fb_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])


In [92]:
#rename clusters based on velo percentile
df_right_fb_velo['Cluster'] = df_right_fb_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1858586982.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_fb_velo['Cluster'] = df_right_fb_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})


In [93]:
#rename cluster column to Pitch Velocity description
df_right_fb_velo = df_right_fb_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [94]:
#replace Player Name ID with Player Name from df_right_fb
df_right_fb_velo = df_right_fb_velo.replace({'Player Name ID':df_right_fb['Player Name']})

In [95]:
#combine movement, release, and velo dataframes for right handed fastballs based on Player Name ID
df_right_fb_movement_release_velo = pd.merge(df_right_fb_movement_release, df_right_fb_velo, on='Player Name ID', how='outer')

In [96]:
#df_right_fb_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_right_fb_movement_release_velo = df_right_fb_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]


In [97]:
df_right_fb_movement_release_velo

,Player Name ID,Pitch Velocity,Pitch Movement,Pitch Release
0,"Camarda, Brad",Very Slow,Cut-Ride,OverTheTop
1,"26, Player",Very Slow,Slight-Ride,OverTheTop
2,"26, Player",Slow,Slight-Ride,OverTheTop
3,"26, Player",Very Slow,Slight-Ride,3/4
4,"26, Player",Slow,Slight-Ride,3/4
...,...,...,...,...
26409,"palmer, blaine",Slow,Slight Sink,3/4
26410,"palmer, blaine",Slow,Slight Sink,3/4
26411,"palmer, blaine",Slow,Slight Sink,3/4
26412,"palmer, blaine",Slow,Slight Sink,3/4


In [98]:
# rename Player Name ID to Player Name in df_right_fb_movement_release_velo
df_right_fb_movement_release_velo = df_right_fb_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [99]:
#df_right_fb2
#df_right_fb only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_right_fb2 = df_right_fb[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]

In [100]:
df_right_fb2

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension
0,"Camarda, Brad",Right,86.058372,20.857697,7.826656,5.968720,1.035166,6.131466
1,"26, Player",Right,88.225560,14.189920,11.144770,5.796032,1.503844,5.704834
2,"26, Player",Right,89.971780,11.069930,17.108043,5.671890,1.488883,5.735810
3,"31, Player",Right,86.911501,21.583298,12.008329,6.576148,1.460385,5.292216
4,"31, Player",Right,87.402110,17.496510,15.999810,6.614730,1.470840,5.112000
...,...,...,...,...,...,...,...,...
7707,"ketchin, mac",Right,86.834054,17.234635,12.482216,5.443459,2.799271,6.973298
7708,"ketchin, mac",Right,86.973734,11.559556,16.166323,5.307880,2.894480,6.785321
7709,"palmer, blaine",Right,90.345730,15.297171,8.149070,5.864502,2.232665,5.782136
7710,"palmer, blaine",Right,90.174823,11.892075,11.506785,5.848382,2.374415,5.712340


In [101]:
#combine df_right_fb2 and df_right_fb_movement_release_velo if name appears in both dataframes
df_right_fb_final = pd.merge(df_right_fb2, df_right_fb_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_right_fb_final
df_right_fb_final = df_right_fb_final.drop_duplicates(subset=['Player Name'])

In [102]:
#sort high to low by velocity
df_right_fb_final = df_right_fb_final.sort_values(by=['Velocity'], ascending=False)

In [103]:
df_right_fb_final.head()

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release
12699,"Duran, Jhoan",Right,100.784496,14.055116,11.666744,6.075640,1.969729,6.219961,Very Hard,Slight-Ride,3/4
33011,"Muñoz, Andrés",Right,100.259241,13.343089,12.269165,5.297873,1.971570,6.356456,Very Hard,Slight Sink,3/4
20778,"Hicks, Jordan",Right,100.050000,10.700000,7.760000,6.141667,2.120000,6.650000,Very Hard,Cut-Ride,3/4
8590,"Clase, Emmanuel",Right,99.945000,13.404000,4.290000,6.098000,0.684000,6.460000,Very Hard,Cut-Ride,OverTheTop
31987,"Montes de Oca, Bryce",Right,99.850000,10.176000,15.810000,5.486000,1.304000,7.000000,Very Hard,Slight Sink,3/4


In [104]:
######BEGIN SLIDER##############################################################################################

In [105]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_slider_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_slider_movement['Cluster'] = movement_labels

In [106]:
#summary of clusters
df_right_slider_movement.groupby('Cluster').mean()

In [107]:
#value counts by cluster
df_right_slider_movement['Cluster'].value_counts()

In [108]:
#rename clusters based on pitch description
df_right_slider_movement['Cluster'] = df_right_slider_movement['Cluster'].replace({0:'Gyro',1:'Sweeper',2:'Gyro',3:'Cutter'})

In [109]:
df_right_slider_movement

In [110]:
#Change the name of cluster column to Pitch Movement
df_right_slider_movement = df_right_slider_movement.rename(columns={'Cluster':'Pitch Movement'})

In [111]:
#replace Player Name ID with Player Name from df_right_slider
df_right_slider_movement = df_right_slider_movement.replace({'Player Name ID':df_right_slider['Player Name']})

In [112]:
#export to csv
df_right_slider_movement.to_csv('2022_right_slider_movement.csv')

In [113]:
#import csv
df_right_slider_movement = pd.read_csv('2022_right_slider_movement.csv')

In [114]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_slider_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_slider_release['Cluster'] = movement_labels

In [115]:
#summary of clusters
df_right_slider_release.groupby('Cluster').mean()

In [116]:
#count of clusters
df_right_slider_release['Cluster'].value_counts()

In [117]:
#rename clusters based on Release description
df_right_slider_release['Cluster'] = df_right_slider_release['Cluster'].replace({0:'Low 3/4 | Sidearm',1:'3/4',2:'3/4',3:'OverTheTop'})

In [118]:
#Change the name of cluster column to Pitch Release
df_right_slider_release = df_right_slider_release.rename(columns={'Cluster':'Pitch Release'})

In [119]:
#replace Player Name ID with Player Name from df_right_slider
df_right_slider_release = df_right_slider_release.replace({'Player Name ID':df_right_slider['Player Name']})

In [120]:
#export to csv
df_right_slider_release.to_csv('2022_right_slider_release.csv')

In [121]:
#import csv
df_right_slider_release = pd.read_csv('2022_right_slider_release.csv')

In [122]:
#combine movement and release dataframes for right handed sliders based on Player Name ID
df_right_slider_movement_release = pd.merge(df_right_slider_movement, df_right_slider_release, on='Player Name ID', how='outer')

In [123]:
df_right_slider_movement_release

,Unnamed: 0_x,Vertical Break,Horizontal Break,Player Name ID,Pitch Movement,Unnamed: 0_y,Release Height,Side Release,Pitch Release
0,0,4.909138,-3.336860,"Camarda, Brad",Gyro,0,5.906344,1.204252,3/4
1,1,-0.013320,-5.304713,"26, Player",Gyro,1,5.531040,1.683390,3/4
2,2,6.824836,0.915253,"31, Player",Gyro,2,6.392824,1.704149,3/4
3,3,6.781780,1.957320,"Abbott, Alex",Gyro,3,4.944750,1.546980,3/4
4,4,5.941834,10.268646,"Abbott, Cory",Cutter,4,5.875895,2.473057,3/4
...,...,...,...,...,...,...,...,...,...
3977,3977,-2.344615,-7.107692,"Zuñiga, Guillermo",Gyro,3977,5.813077,2.591538,3/4
3978,3978,0.601730,6.647170,"Zwitzer, Jurgen",Gyro,3978,5.945320,1.694800,3/4
3979,3979,5.297064,-3.603578,"deGrom, Jacob",Gyro,3979,5.653188,1.462133,3/4
3980,3980,0.713606,-7.000836,"ketchin, mac",Gyro,3980,5.363561,2.868857,3/4


In [124]:
#remove Unnamed: 0 column
df_right_slider_movement_release = df_right_slider_movement_release.drop(['Unnamed: 0_x'], axis=1)
df_right_slider_movement_release = df_right_slider_movement_release.drop(['Unnamed: 0_y'], axis=1)

In [125]:
df_right_slider_movement_release

,Vertical Break,Horizontal Break,Player Name ID,Pitch Movement,Release Height,Side Release,Pitch Release
0,4.909138,-3.336860,"Camarda, Brad",Gyro,5.906344,1.204252,3/4
1,-0.013320,-5.304713,"26, Player",Gyro,5.531040,1.683390,3/4
2,6.824836,0.915253,"31, Player",Gyro,6.392824,1.704149,3/4
3,6.781780,1.957320,"Abbott, Alex",Gyro,4.944750,1.546980,3/4
4,5.941834,10.268646,"Abbott, Cory",Cutter,5.875895,2.473057,3/4
...,...,...,...,...,...,...,...
3977,-2.344615,-7.107692,"Zuñiga, Guillermo",Gyro,5.813077,2.591538,3/4
3978,0.601730,6.647170,"Zwitzer, Jurgen",Gyro,5.945320,1.694800,3/4
3979,5.297064,-3.603578,"deGrom, Jacob",Gyro,5.653188,1.462133,3/4
3980,0.713606,-7.000836,"ketchin, mac",Gyro,5.363561,2.868857,3/4


In [126]:
df_right_slider_velo['Cluster'] = pd.cut(df_right_slider_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1088555441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_slider_velo['Cluster'] = pd.cut(df_right_slider_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])


In [127]:
#rename clusters based on velo percentile
df_right_slider_velo['Cluster'] = df_right_slider_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/2365456233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_slider_velo['Cluster'] = df_right_slider_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})


In [128]:
#rename cluster column to Pitch Velocity description
df_right_slider_velo = df_right_slider_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [129]:
#replace Player Name ID with Player Name from df_right_slider
df_right_slider_velo = df_right_slider_velo.replace({'Player Name ID':df_right_slider['Player Name']})

In [130]:
#combine movement, release, and velo dataframes for right handed sliders based on Player Name ID
df_right_slider_movement_release_velo = pd.merge(df_right_slider_movement_release, df_right_slider_velo, on='Player Name ID', how='outer')

In [131]:
#df_right_slider_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_right_slider_movement_release_velo = df_right_slider_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]

In [132]:
# rename Player Name ID to Player Name in df_right_slider_movement_release_velo
df_right_slider_movement_release_velo = df_right_slider_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [133]:
#df_right_slider2
#df_right_slider only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_right_slider2 = df_right_slider[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]


In [134]:
#combine df_right_fb2 and df_right_fb_movement_release_velo if name appears in both dataframes
df_right_slider_final = pd.merge(df_right_slider2, df_right_slider_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_right_fb_final
df_right_slider_final = df_right_slider_final.drop_duplicates(subset=['Player Name'])

In [135]:
df_right_slider_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release
0,"Camarda, Brad",Right,79.134562,4.909138,-3.336860,5.906344,1.204252,5.886594,Very Slow,Gyro,3/4
1,"26, Player",Right,80.292060,-0.013320,-5.304713,5.531040,1.683390,5.096890,Slow,Gyro,3/4
2,"31, Player",Right,78.287349,6.824836,0.915253,6.392824,1.704149,5.334070,Very Slow,Gyro,3/4
3,"Abbott, Alex",Right,77.190690,6.781780,1.957320,4.944750,1.546980,5.603620,Very Slow,Gyro,3/4
4,"Abbott, Cory",Right,86.606114,5.941834,10.268646,5.875895,2.473057,6.065066,Very Hard,Cutter,3/4
...,...,...,...,...,...,...,...,...,...,...,...
3977,"Zuñiga, Guillermo",Right,84.392308,-2.344615,-7.107692,5.813077,2.591538,6.176923,Hard,Gyro,3/4
3978,"Zwitzer, Jurgen",Right,77.870060,0.601730,6.647170,5.945320,1.694800,7.148120,Very Slow,Gyro,3/4
3979,"deGrom, Jacob",Right,92.505505,5.297064,-3.603578,5.653188,1.462133,6.601376,Very Hard,Gyro,3/4
3980,"ketchin, mac",Right,78.943380,0.713606,-7.000836,5.363561,2.868857,6.439231,Very Slow,Gyro,3/4


In [136]:
#BEGIN CHANGEUP######################################################################################################

In [137]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_changeup_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_changeup_movement['Cluster'] = movement_labels

In [138]:
df_right_changeup_movement

In [139]:
#summary of clusters
df_right_changeup_movement.groupby('Cluster').mean()

In [140]:
#value counts by cluster
df_right_changeup_movement['Cluster'].value_counts()

In [141]:
#rename clusters based on pitch description
df_right_changeup_movement['Cluster'] = df_right_changeup_movement['Cluster'].replace({0:'FB Shape',1:'Depth w/ Fade',2:'FB Shape',3:'Depth w/ fade',4:'Cut-Change'})

In [142]:
#change the name of cluster column to Pitch Movement
df_right_changeup_movement = df_right_changeup_movement.rename(columns={'Cluster':'Pitch Movement'})

In [143]:
#replace Player Name ID with Player Name from df_right_changeup
df_right_changeup_movement = df_right_changeup_movement.replace({'Player Name ID':df_right_changeup['Player Name']})

In [144]:
#export to csv
df_right_changeup_movement.to_csv('2022_right_changeup_movement.csv')


In [145]:
#import csv
df_right_changeup_movement = pd.read_csv('2022_right_changeup_movement.csv')

In [146]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_changeup_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_changeup_release['Cluster'] = movement_labels

In [147]:
#summary of clusters
df_right_changeup_release.groupby('Cluster').mean()

In [148]:
#count of clusters
df_right_changeup_release['Cluster'].value_counts()

In [149]:
#rename clusters based on Release description
df_right_changeup_release['Cluster'] = df_right_changeup_release['Cluster'].replace({0:'3/4',1:'3/4',2:'Low 3/4 | Sidearm',3:'OverTheTop'})

In [150]:
#Change the name of cluster column to Pitch Release
df_right_changeup_release = df_right_changeup_release.rename(columns={'Cluster':'Pitch Release'})

In [151]:
#replace Player Name ID with Player Name from df_right_changeup
df_right_changeup_release = df_right_changeup_release.replace({'Player Name ID':df_right_changeup['Player Name']})

In [152]:
#export to csv
df_right_changeup_release.to_csv('2022_right_changeup_release.csv')

In [153]:
#import csv
df_right_changeup_release = pd.read_csv('2022_right_changeup_release.csv')

In [154]:
#combine movement and release dataframes for right handed changeups based on Player Name ID
df_right_changeup_movement_release = pd.merge(df_right_changeup_movement, df_right_changeup_release, on='Player Name ID', how='outer')

In [155]:
df_right_changeup_velo['Cluster'] = pd.cut(df_right_changeup_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/903643330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_changeup_velo['Cluster'] = pd.cut(df_right_changeup_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])


In [156]:
#rename clusters based on velo percentile
df_right_changeup_velo['Cluster'] = df_right_changeup_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/2225027042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_changeup_velo['Cluster'] = df_right_changeup_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})


In [157]:
#rename cluster column to Pitch Velocity description
df_right_changeup_velo = df_right_changeup_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [158]:
#replace Player Name ID with Player Name from df_right_changeup
df_right_changeup_velo = df_right_changeup_velo.replace({'Player Name ID':df_right_changeup['Player Name']})

In [159]:
#combine movement, release, and velo dataframes for right handed changeups based on Player Name ID
df_right_changeup_movement_release_velo = pd.merge(df_right_changeup_movement_release, df_right_changeup_velo, on='Player Name ID', how='outer')

In [160]:
#df_right_changeup_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_right_changeup_movement_release_velo = df_right_changeup_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]

In [161]:
# rename Player Name ID to Player Name in df_right_changeup_movement_release_velo
df_right_changeup_movement_release_velo = df_right_changeup_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [162]:
#df_right_changeup2
#df_right_changeup only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_right_changeup2 = df_right_changeup[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]

In [163]:
#combine df_right_changeup2 and df_right_changeup_movement_release_velo if name appears in both dataframes
df_right_changeup_final = pd.merge(df_right_changeup2, df_right_changeup_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_right_changeup_final
df_right_changeup_final = df_right_changeup_final.drop_duplicates(subset=['Player Name'])

In [164]:
#sort high to low by velocity
df_right_changeup_final = df_right_changeup_final.sort_values(by=['Velocity'], ascending=False)


In [165]:
df_right_changeup_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release
1387,"Gil, Luis",Right,92.640000,8.000000,15.264000,5.432667,2.670667,6.860000,Very Hard,Depth w/ Fade,3/4
531,"Cabrera, Edward",Right,92.438265,8.045510,16.084286,6.236684,1.715179,6.515051,Very Hard,Depth w/ Fade,3/4
614,"Castro, Miguel",Right,92.421818,0.703636,17.501455,5.008000,2.335545,6.699091,Very Hard,Depth w/ Fade,3/4
4159,"Wiseman, Kevin",Right,92.114450,22.871845,7.005355,6.044660,0.813585,5.824335,Very Hard,FB Shape,OverTheTop
1725,"Hicks, Jordan",Right,92.110714,8.365714,15.510000,5.790357,2.956786,6.621429,Very Hard,Depth w/ Fade,3/4
...,...,...,...,...,...,...,...,...,...,...,...
4013,"Washington, Brett",Right,67.715291,12.345447,8.310871,5.166408,2.516082,4.348706,Very Slow,FB Shape,3/4
193,"Bargsley, Brannon",Right,64.629437,12.925508,5.147927,6.055575,2.687242,5.101897,Very Slow,FB Shape,3/4
957,"Denton, Cade",Right,60.942345,10.987396,13.804407,5.415439,2.171211,5.407529,Very Slow,FB Shape,3/4
918,"DeJean, Chase",Right,57.204350,15.076190,26.529270,6.195920,2.696910,5.614170,Very Slow,Depth w/ Fade,3/4


In [166]:
#BEGIN CURVEBALL######################################################################################################

In [167]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_curveball_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=3)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_curveball_movement['Cluster'] = movement_labels

In [168]:
#summary of clusters
df_right_curveball_movement.groupby('Cluster').mean()

In [169]:
#value counts by cluster
df_right_curveball_movement['Cluster'].value_counts()

In [170]:
#rename clusters based on pitch description
df_right_curveball_movement['Cluster'] = df_right_curveball_movement['Cluster'].replace({0:'Small Curve',1:'True Curve',2:'Slider Type'})

In [171]:
#change the name of cluster column to Pitch Movement
df_right_curveball_movement = df_right_curveball_movement.rename(columns={'Cluster':'Pitch Movement'})

In [172]:
#replace Player Name ID with Player Name from df_right_curveball
df_right_curveball_movement = df_right_curveball_movement.replace({'Player Name ID':df_right_curveball['Player Name']})

In [173]:
#export to csv
df_right_curveball_movement.to_csv('2022_right_curveball_movement.csv')

In [174]:
#import csv
df_right_curveball_movement = pd.read_csv('2022_right_curveball_movement.csv')

In [175]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_right_curveball_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_right_curveball_release['Cluster'] = movement_labels

In [176]:
#summary of clusters
df_right_curveball_release.groupby('Cluster').mean()

In [177]:
#value counts by cluster
df_right_curveball_release['Cluster'].value_counts()

In [178]:
#rename clusters based on Release description
df_right_curveball_release['Cluster'] = df_right_curveball_release['Cluster'].replace({0:'3/4',1:'3/4',2:'Low 3/4 | Sidearm',3:'OverTheTop'})

In [179]:
#Change the name of cluster column to Pitch Release
df_right_curveball_release = df_right_curveball_release.rename(columns={'Cluster':'Pitch Release'})

In [180]:
#replace Player Name ID with Player Name from df_right_changeup
df_right_curveball_release = df_right_curveball_release.replace({'Player Name ID':df_right_curveball['Player Name']})

In [181]:
#export to csv
df_right_curveball_release.to_csv('2022_right_curveball_release.csv')

In [182]:
#import csv
df_right_curveball_release = pd.read_csv('2022_right_curveball_release.csv')

In [183]:
#combine movement and release dataframes for right handed changeups based on Player Name ID
df_right_curveball_movement_release = pd.merge(df_right_curveball_movement, df_right_curveball_release, on='Player Name ID', how='outer')

In [184]:
df_right_curveball_velo['Cluster'] = pd.cut(df_right_curveball_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3426345271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_curveball_velo['Cluster'] = pd.cut(df_right_curveball_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])


In [185]:
#rename clusters based on velo percentile
df_right_curveball_velo['Cluster'] = df_right_curveball_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1490686396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_curveball_velo['Cluster'] = df_right_curveball_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})


In [186]:
#rename cluster column to Pitch Velocity description
df_right_curveball_velo = df_right_curveball_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [187]:
#replace Player Name ID with Player Name from df_right_curveball
df_right_curveball_velo = df_right_curveball_velo.replace({'Player Name ID':df_right_curveball['Player Name']})

In [188]:
#combine movement, release, and velo dataframes for right handed curveballs based on Player Name ID
df_right_curveball_movement_release_velo = pd.merge(df_right_curveball_movement_release, df_right_curveball_velo, on='Player Name ID', how='outer')

In [189]:
#df_right_curveball_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_right_curveball_movement_release_velo = df_right_curveball_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]

In [190]:
#player Name ID to Player Name in df_right_changeup_movement_release_velo
df_right_curveball_movement_release_velo = df_right_curveball_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [191]:
#df_right_curveball2
#df_right_curveball only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_right_curveball2 = df_right_curveball[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]

In [192]:
#combine df_right_curveball2 and df_right_curveball_movement_release_velo if name appears in both dataframes
df_right_curveball_final = pd.merge(df_right_curveball2, df_right_curveball_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_right_changeup_final
df_right_curveball_final = df_right_curveball_final.drop_duplicates(subset=['Player Name'])

In [193]:
df_right_curveball_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release
0,"Camarda, Brad",Right,75.551580,1.912192,-2.619799,5.918850,1.299285,5.641056,Slow,Slider Type,3/4
1,", Wyatt",Right,70.429887,1.788356,-12.910680,5.216501,2.634507,5.415914,Very Slow,Slider Type,3/4
2,"26, Player",Right,79.335070,-3.607038,-8.547632,5.604442,1.732678,5.017418,Hard,Slider Type,3/4
3,"31, Player",Right,72.829933,-2.431298,-3.293793,6.343700,1.767802,5.003240,Very Slow,Slider Type,3/4
4,"Abbott, Alex",Right,73.345997,-2.250720,-14.100376,4.771580,1.606376,5.844812,Very Slow,True Curve,3/4
...,...,...,...,...,...,...,...,...,...,...,...
4200,"Zwitzer, Jurgen",Right,74.087646,-6.187847,-5.536039,6.070389,1.730360,6.438654,Very Slow,Small Curve,3/4
4201,"deGrom, Jacob",Right,83.409091,-2.572364,-7.169455,5.768364,1.445818,6.550909,Very Hard,Small Curve,3/4
4202,"herring, jackson",Right,66.924280,4.325315,-14.382062,3.631639,4.159856,6.061304,Very Slow,Small Curve,Low 3/4 | Sidearm
4203,"ketchin, mac",Right,77.795967,-1.526858,-5.762332,5.310678,2.880815,6.542977,Average,Small Curve,3/4


In [194]:
# create a new column in df_right_fb_final called Pitch Type and set it equal to FB
df_right_fb_final['Pitch Type'] = 'FB'
df_right_slider_final['Pitch Type'] = 'SL'
df_right_changeup_final['Pitch Type'] = 'CH'
df_right_curveball_final['Pitch Type'] = 'CB'

In [195]:
#combine all 4 dataframes into one
df_right_final = pd.concat([df_right_fb_final, df_right_slider_final, df_right_changeup_final, df_right_curveball_final])

In [196]:
df_right_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release,Pitch Type
12699,"Duran, Jhoan",Right,100.784496,14.055116,11.666744,6.075640,1.969729,6.219961,Very Hard,Slight-Ride,3/4,FB
33011,"Muñoz, Andrés",Right,100.259241,13.343089,12.269165,5.297873,1.971570,6.356456,Very Hard,Slight Sink,3/4,FB
20778,"Hicks, Jordan",Right,100.050000,10.700000,7.760000,6.141667,2.120000,6.650000,Very Hard,Cut-Ride,3/4,FB
8590,"Clase, Emmanuel",Right,99.945000,13.404000,4.290000,6.098000,0.684000,6.460000,Very Hard,Cut-Ride,OverTheTop,FB
31987,"Montes de Oca, Bryce",Right,99.850000,10.176000,15.810000,5.486000,1.304000,7.000000,Very Hard,Slight Sink,3/4,FB
...,...,...,...,...,...,...,...,...,...,...,...,...
4200,"Zwitzer, Jurgen",Right,74.087646,-6.187847,-5.536039,6.070389,1.730360,6.438654,Very Slow,Small Curve,3/4,CB
4201,"deGrom, Jacob",Right,83.409091,-2.572364,-7.169455,5.768364,1.445818,6.550909,Very Hard,Small Curve,3/4,CB
4202,"herring, jackson",Right,66.924280,4.325315,-14.382062,3.631639,4.159856,6.061304,Very Slow,Small Curve,Low 3/4 | Sidearm,CB
4203,"ketchin, mac",Right,77.795967,-1.526858,-5.762332,5.310678,2.880815,6.542977,Average,Small Curve,3/4,CB


In [197]:
#sort Player Name alphabetically
df_right_final = df_right_final.sort_values(by=['Player Name'], ascending=True)

In [198]:
df_right_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release,Pitch Type
0,"Camarda, Brad",Right,86.058372,20.857697,7.826656,5.968720,1.035166,6.131466,Very Slow,Cut-Ride,OverTheTop,FB
0,"Camarda, Brad",Right,77.085644,16.007712,13.758622,5.916989,1.375978,6.064864,Very Slow,FB Shape,3/4,CH
0,"Camarda, Brad",Right,79.134562,4.909138,-3.336860,5.906344,1.204252,5.886594,Very Slow,Gyro,3/4,SL
0,"Camarda, Brad",Right,75.551580,1.912192,-2.619799,5.918850,1.299285,5.641056,Slow,Slider Type,3/4,CB
1,", Wyatt",Right,79.780816,11.211545,18.516320,5.188266,2.806513,6.063058,Very Slow,Depth w/ Fade,3/4,CH
...,...,...,...,...,...,...,...,...,...,...,...,...
4259,"ketchin, mac",Right,86.008903,12.854712,15.881748,5.405018,2.849353,6.908312,Hard,FB Shape,3/4,CH
51333,"palmer, blaine",Right,90.345730,15.297171,8.149070,5.864502,2.232665,5.782136,Slow,Ride,3/4,FB
3981,"palmer, blaine",Right,82.802023,5.935621,-4.954704,5.961693,2.237664,5.405794,Average,Gyro,3/4,SL
4204,"palmer, blaine",Right,75.199820,-9.050802,-16.506743,5.812135,2.126050,5.364642,Slow,Small Curve,3/4,CB


In [199]:
#new dataframe with description
df_right_final_description = df_right_final[['Player Name','Throws','Pitch Type','Pitch Velocity','Pitch Movement','Pitch Release']]

In [200]:
#combine columns Pitch Velocity','Pitch Movement','Pitch Release' into one column called Pitch Description
df_right_final_description['Pitch Description'] = df_right_final_description['Pitch Velocity'].astype(str) + ' ' + df_right_final_description['Pitch Movement'].astype(str) + ' ' + df_right_final_description['Pitch Release'].astype(str)

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/883310487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_right_final_description['Pitch Description'] = df_right_final_description['Pitch Velocity'].astype(str) + ' ' + df_right_final_description['Pitch Movement'].astype(str) + ' ' + df_right_final_description['Pitch Release'].astype(str)


In [201]:
#only include Player Name, Throws, Pitch Type, Pitch Description
df_right_final_description = df_right_final_description[['Player Name','Throws','Pitch Type','Pitch Description']]

In [202]:
#locate Jacob deGrom
df_right_final_description.loc[df_right_final_description['Player Name'] == 'Skenes, Paul']

,Player Name,Throws,Pitch Type,Pitch Description
3279,"Skenes, Paul",Right,SL,Hard Gyro 3/4
3497,"Skenes, Paul",Right,CH,Very Hard Depth w/ fade 3/4
3458,"Skenes, Paul",Right,CB,Very Hard Small Curve 3/4
42206,"Skenes, Paul",Right,FB,Very Hard Slight Sink 3/4


In [203]:
###BEGIN LEFTY FB###################################################################################

In [204]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_fb_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_fb_movement['Cluster'] = movement_labels

In [205]:
#summary of clusters
df_left_fb_movement.groupby('Cluster').mean()

In [206]:
#value counts by cluster
df_left_fb_movement['Cluster'].value_counts()

In [207]:
#rename clusters based on pitch description
df_left_fb_movement['Cluster'] = df_left_fb_movement['Cluster'].replace({0:'Sink',1:'Cut-Ride ',2:'Slight Sink',3:'Ride',4:'Slight Ride'})

In [208]:
#Change the name of cluster column to Pitch Movement
df_left_fb_movement = df_left_fb_movement.rename(columns={'Cluster':'Pitch Movement'})

In [209]:
#replace Player Name ID with Player Name from df_left_fb
df_left_fb_movement = df_left_fb_movement.replace({'Player Name ID':df_left_fb['Player Name']})

In [210]:
#export to csv
df_left_fb_movement.to_csv('2022_left_fb_movement.csv')

In [211]:
#import csv
df_left_fb_movement = pd.read_csv('2022_left_fb_movement.csv')

In [212]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data2 = scaler.fit_transform(df_left_fb_release)

# Implement KMeans
kmeans = KMeans(n_clusters=5)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data2)


# Add the labels to your dataframe
df_left_fb_release['Cluster'] = movement_labels

In [213]:
#summary of clusters
df_left_fb_release.groupby('Cluster').mean()

In [214]:
#count of clusters
df_left_fb_release['Cluster'].value_counts()

In [215]:
#rename clusters based on Release description
df_left_fb_release['Cluster'] = df_left_fb_release['Cluster'].replace({0:'3/4',1:'OverTheTop',2:'Low 3/4 | Sidearm',3:'3/4',4:'OverTheTop'})

In [216]:
#Change the name of cluster column to Pitch Release
df_left_fb_release = df_left_fb_release.rename(columns={'Cluster':'Pitch Release'})

In [217]:
#replace Player Name ID with Player Name from df_left_fb
df_left_fb_release = df_left_fb_release.replace({'Player Name ID':df_left_fb['Player Name']})

In [218]:
#export to csv
df_left_fb_release.to_csv('2022_left_fb_release.csv')

In [219]:
#import csv
df_left_fb_release = pd.read_csv('2022_left_fb_release.csv')

In [220]:
#combine movement and release dataframes for left handed fastballs based on Player Name ID
df_left_fb_movement_release = pd.merge(df_left_fb_movement, df_left_fb_release, on='Player Name ID', how='outer')


In [221]:
#df_left_fb_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_left_fb_velo['Cluster'] = pd.cut(df_left_fb_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3599992626.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_fb_velo['Cluster'] = pd.cut(df_left_fb_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])


In [222]:
#rename clusters based on velo percentile
df_left_fb_velo['Cluster'] = df_left_fb_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3936579694.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_fb_velo['Cluster'] = df_left_fb_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})


In [223]:
#rename cluster column to Pitch Velocity description
df_left_fb_velo = df_left_fb_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [224]:
#replace Player Name ID with Player Name from df_left_fb
df_left_fb_velo = df_left_fb_velo.replace({'Player Name ID':df_left_fb['Player Name']})

In [225]:
#combine movement, release, and velo dataframes for left handed fastballs based on Player Name ID
df_left_fb_movement_release_velo = pd.merge(df_left_fb_movement_release, df_left_fb_velo, on='Player Name ID', how='outer')

In [226]:
#df_left_fb_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_left_fb_movement_release_velo = df_left_fb_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]

In [227]:
# rename Player Name ID to Player Name in df_left_fb_movement_release_velo
df_left_fb_movement_release_velo = df_left_fb_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [228]:
#df_left_fb2
#df_left_fb only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_left_fb2 = df_left_fb[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]

In [229]:
#combine df_left_fb2 and df_left_fb_movement_release_velo if name appears in both dataframes
df_left_fb_final = pd.merge(df_left_fb2, df_left_fb_movement_release_velo, on='Player Name', how='inner')
#delete duplicate rows if name appears in more than once in df_left_fb_final
df_left_fb_final = df_left_fb_final.drop_duplicates(subset=['Player Name'])


In [230]:
#sort high to low by velocity
df_left_fb_final = df_left_fb_final.sort_values(by=['Velocity'], ascending=False)

In [231]:
df_left_fb_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release
1749,"Brown, Daniel",Left,98.881960,12.895436,-18.410347,6.338786,-1.284695,5.872700,Very Hard,Slight Sink,3/4
14441,"Soto, Gregory",Left,98.666289,13.670142,-11.156941,5.507535,-2.182918,6.671955,Very Hard,Slight Ride,3/4
251,"Alvarado, José",Left,98.075000,16.404000,-5.082000,6.540500,-0.751500,6.890000,Very Hard,Ride,OverTheTop
6530,"Hader, Josh",Left,97.533007,17.700084,-8.553902,5.528112,-2.836951,6.776783,Very Hard,Ride,Low 3/4 | Sidearm
2639,"Chapman, Aroldis",Left,97.389409,17.806108,-4.460099,6.070961,-0.690887,7.112808,Very Hard,Ride,OverTheTop
...,...,...,...,...,...,...,...,...,...,...,...
10765,"Morgan, Tanner",Left,82.070640,14.958390,-10.963630,5.516790,-1.824000,5.425360,Very Slow,Slight Ride,3/4
17244,"mercado, austin",Left,82.001497,16.184380,-6.877587,5.255820,-1.735437,6.571337,Very Slow,Cut-Ride,3/4
16,"Rue, Ryan",Left,81.998540,17.643280,-8.789530,5.776840,-1.657960,4.799420,Very Slow,Ride,3/4
11089,"Nicholson, Gabe",Left,81.411830,16.602690,-0.290980,6.184000,-0.683920,5.858100,Very Slow,Cut-Ride,OverTheTop


In [232]:
######BEGIN SLIDER##############################################################################################

In [233]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_slider_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_slider_movement['Cluster'] = movement_labels

In [234]:
#summary of clusters
df_left_slider_movement.groupby('Cluster').mean()

In [235]:
#value counts by cluster
df_left_slider_movement['Cluster'].value_counts()

In [236]:
#rename clusters based on pitch description
df_left_slider_movement['Cluster'] = df_left_slider_movement['Cluster'].replace({0:'Gyro',1:'Cutter',2:'Gyro',3:'Sweeper'})

In [237]:
#change the name of cluster column to Pitch Movement
df_left_slider_movement = df_left_slider_movement.rename(columns={'Cluster':'Pitch Movement'})

In [238]:
#replace Player Name ID with Player Name from df_left_slider
df_left_slider_movement = df_left_slider_movement.replace({'Player Name ID':df_left_slider['Player Name']})

In [239]:
#export to csv
df_left_slider_movement.to_csv('2022_left_slider_movement.csv')

In [240]:
#import csv
df_left_slider_movement = pd.read_csv('2022_left_slider_movement.csv')

In [241]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_slider_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_slider_release['Cluster'] = movement_labels

In [242]:
#summary of clusters
df_left_slider_release.groupby('Cluster').mean()

In [243]:
#count of clusters
df_left_slider_release['Cluster'].value_counts()

In [244]:
#rename clusters based on Release description
df_left_slider_release['Cluster'] = df_left_slider_release['Cluster'].replace({0:'3/4',1:'Low 3/4 | Sidearm',2:'OverTheTop',3:'3/4'})

In [245]:
#Change the name of cluster column to Pitch Release
df_left_slider_release = df_left_slider_release.rename(columns={'Cluster':'Pitch Release'})

In [246]:
#replace Player Name ID with Player Name from df_left_slider
df_left_slider_release = df_left_slider_release.replace({'Player Name ID':df_left_slider['Player Name']})

In [247]:
#export to csv
df_left_slider_release.to_csv('2022_left_slider_release.csv')


In [248]:
#import csv
df_left_slider_release = pd.read_csv('2022_left_slider_release.csv')

In [249]:
#combine movement and release dataframes for left handed sliders based on Player Name ID
df_left_slider_movement_release = pd.merge(df_left_slider_movement, df_left_slider_release, on='Player Name ID', how='outer')

In [250]:
#df_left_slider_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_left_slider_velo['Cluster'] = pd.cut(df_left_slider_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3797296620.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_slider_velo['Cluster'] = pd.cut(df_left_slider_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])


In [251]:
#rename clusters based on velo percentile
df_left_slider_velo['Cluster'] = df_left_slider_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3005059942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_slider_velo['Cluster'] = df_left_slider_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})


In [252]:
#rename cluster column to Pitch Velocity description
df_left_slider_velo = df_left_slider_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [253]:
#replace Player Name ID with Player Name from df_left_slider
df_left_slider_velo = df_left_slider_velo.replace({'Player Name ID':df_left_slider['Player Name']})

In [254]:
#combine movement, release, and velo dataframes for left handed sliders based on Player Name ID
df_left_slider_movement_release_velo = pd.merge(df_left_slider_movement_release, df_left_slider_velo, on='Player Name ID', how='outer')

In [255]:
#df_left_slider_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_left_slider_movement_release_velo = df_left_slider_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]
# rename Player Name ID to Player Name in df_left_slider_movement_release_velo
df_left_slider_movement_release_velo = df_left_slider_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})


In [256]:
#df_left_slider2
#df_left_slider only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_left_slider2 = df_left_slider[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]
#combine df_left_slider2 and df_left_slider_movement_release_velo if name appears in both dataframes
df_left_slider_final = pd.merge(df_left_slider2, df_left_slider_movement_release_velo, on='Player Name', how='inner')

In [257]:
#delete duplicate rows if name appears in more than once in df_left_slider_final
df_left_slider_final = df_left_slider_final.drop_duplicates(subset=['Player Name'])

In [258]:
#sort high to low by velocity
df_left_slider_final = df_left_slider_final.sort_values(by=['Velocity'], ascending=False)

In [259]:
df_left_slider_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release
708,"McClanahan, Shane",Left,89.349610,3.739948,3.599688,6.066805,-1.062104,6.802597,Very Hard,Gyro,3/4
1029,"Soto, Gregory",Left,89.304264,6.149302,1.144651,5.540465,-2.152868,6.725581,Very Hard,Gyro,3/4
697,"Matz, Steven",Left,89.183333,7.316000,-1.552000,5.716000,-1.977000,6.386667,Very Hard,Cutter,3/4
971,"Scott, Tanner",Left,89.168388,-0.826398,5.228463,5.246625,-2.588904,6.233249,Very Hard,Gyro,Low 3/4 | Sidearm
1013,"Skubal, Tarik",Left,89.160630,7.879937,1.238362,6.098047,-2.498094,6.253858,Very Hard,Cutter,3/4
...,...,...,...,...,...,...,...,...,...,...,...
150,"Buehler, Chris",Left,69.823990,2.740160,4.475320,5.076720,-1.015740,5.616990,Very Slow,Gyro,3/4
684,"Marshalwitz, Cam",Left,69.633175,5.664275,2.036455,5.295010,-1.694660,5.377265,Very Slow,Cutter,Low 3/4 | Sidearm
864,"Powell, Landen",Left,68.032630,8.538810,-1.153557,6.215600,-1.464750,5.918703,Very Slow,Cutter,3/4
549,"Jackson, Dylan",Left,67.996320,5.884140,0.193820,6.302910,0.716310,4.881780,Very Slow,Cutter,OverTheTop


In [260]:
######BEGIN CHANGEUP##############################################################################################

In [261]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_changeup_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_changeup_movement['Cluster'] = movement_labels

In [262]:
#summary of clusters
df_left_changeup_movement.groupby('Cluster').mean()

In [263]:
#value counts by cluster
df_left_changeup_movement['Cluster'].value_counts()

In [264]:
#rename clusters based on pitch description
df_left_changeup_movement['Cluster'] = df_left_changeup_movement['Cluster'].replace({0:'Depth w/ Fade',1:'Drop',2:'FB Shape',3:'Depth w/ fade'})

In [265]:
#change the name of cluster column to Pitch Movement
df_left_changeup_movement = df_left_changeup_movement.rename(columns={'Cluster':'Pitch Movement'})

In [266]:
#replace Player Name ID with Player Name from df_left_changeup
df_left_changeup_movement = df_left_changeup_movement.replace({'Player Name ID':df_left_changeup['Player Name']})

In [267]:
#show all with drop
df_left_changeup_movement.loc[df_left_changeup_movement['Pitch Movement'] == 'Drop']
# delete rows with drop
df_left_changeup_movement = df_left_changeup_movement[df_left_changeup_movement['Pitch Movement'] != 'Drop']


In [268]:
#export to csv
df_left_changeup_movement.to_csv('2022_left_changeup_movement.csv')


In [269]:
#import csv
df_left_changeup_movement = pd.read_csv('2022_left_changeup_movement.csv')

In [270]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_changeup_release)

# Implement KMeans
kmeans = KMeans(n_clusters=4)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_changeup_release['Cluster'] = movement_labels

In [271]:
#summary of clusters
df_left_changeup_release.groupby('Cluster').mean()

In [272]:
# count of clusters
df_left_changeup_release['Cluster'].value_counts()

In [273]:
#rename clusters based on Release description
df_left_changeup_release['Cluster'] = df_left_changeup_release['Cluster'].replace({0:'3/4',1:'3/4',2:'Low 3/4 | Sidearm',3:'OverTheTop'})

In [274]:
#Change the name of cluster column to Pitch Release
df_left_changeup_release = df_left_changeup_release.rename(columns={'Cluster':'Pitch Release'})

In [275]:
#replace Player Name ID with Player Name from df_left_changeup
df_left_changeup_release = df_left_changeup_release.replace({'Player Name ID':df_left_changeup['Player Name']})

In [276]:
#export to csv
df_left_changeup_release.to_csv('2022_left_changeup_release.csv')

In [277]:
#import csv
df_left_changeup_release = pd.read_csv('2022_left_changeup_release.csv')

In [278]:
#combine movement and release dataframes for left handed changeups based on Player Name ID
df_left_changeup_movement_release = pd.merge(df_left_changeup_movement, df_left_changeup_release, on='Player Name ID', how='outer')

In [279]:
#df_left_changeup_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_left_changeup_velo['Cluster'] = pd.cut(df_left_changeup_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/754297321.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_changeup_velo['Cluster'] = pd.cut(df_left_changeup_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])


In [280]:
#rename clusters based on velo percentile
df_left_changeup_velo['Cluster'] = df_left_changeup_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1383753867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_changeup_velo['Cluster'] = df_left_changeup_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})


In [281]:
#rename cluster column to Pitch Velocity description
df_left_changeup_velo = df_left_changeup_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [282]:
#replace Player Name ID with Player Name from df_left_changeup
df_left_changeup_velo = df_left_changeup_velo.replace({'Player Name ID':df_left_changeup['Player Name']})

In [283]:
#combine movement, release, and velo dataframes for left handed changeups based on Player Name ID
df_left_changeup_movement_release_velo = pd.merge(df_left_changeup_movement_release, df_left_changeup_velo, on='Player Name ID', how='outer')


In [284]:
#df_left_changeup_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_left_changeup_movement_release_velo = df_left_changeup_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]
# rename Player Name ID to Player Name in df_left_changeup_movement_release_velo
df_left_changeup_movement_release_velo = df_left_changeup_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [285]:
#df_left_changeup2
#df_left_changeup only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_left_changeup2 = df_left_changeup[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]
#combine df_left_changeup2 and df_left_changeup_movement_release_velo if name appears in both dataframes
df_left_changeup_final = pd.merge(df_left_changeup2, df_left_changeup_movement_release_velo, on='Player Name', how='inner')

In [286]:
#delete duplicate rows if name appears in more than once in df_left_changeup_final
df_left_changeup_final = df_left_changeup_final.drop_duplicates(subset=['Player Name'])

In [287]:
#sort high to low by velocity
df_left_changeup_final = df_left_changeup_final.sort_values(by=['Velocity'], ascending=False)

In [288]:
df_left_changeup_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release
1228,"Soto, Gregory",Left,91.920000,8.984000,-17.048000,5.396000,-2.249333,6.733333,Very Hard,Depth w/ Fade,3/4
285,"Crochet, Garrett",Left,91.375000,12.570000,-13.650000,5.782500,-2.097500,7.425000,Very Hard,FB Shape,3/4
458,"Fry, Paul",Left,91.282051,4.516923,-13.187692,5.146154,-2.486923,6.189744,Very Hard,Depth w/ fade,Low 3/4 | Sidearm
148,"Brentz, Jake",Left,90.837500,4.440000,-17.205000,5.176250,-1.237500,6.750000,Very Hard,Depth w/ fade,3/4
92,"Beeks, Jalen",Left,90.306719,8.401897,-16.614783,6.150711,-0.791660,6.423913,Very Hard,Depth w/ fade,OverTheTop
...,...,...,...,...,...,...,...,...,...,...,...
1021,"Price, Alex",Left,71.611153,11.748415,-13.259724,6.333703,-0.132761,5.276043,Very Slow,Depth w/ Fade,OverTheTop
216,"Cazares, Julian",Left,71.570644,-11.480510,12.124706,5.596686,0.256214,4.665881,Very Slow,NaN,OverTheTop
235,"Claudio, Alex",Left,70.042105,1.509474,-16.282105,4.584737,-2.431579,6.378947,Very Slow,Depth w/ fade,Low 3/4 | Sidearm
42,"Armstrong, Cade",Left,67.723433,-1.307809,-16.139561,3.200834,-3.893133,6.732113,Very Slow,Depth w/ fade,Low 3/4 | Sidearm


In [289]:
######BEGIN CURVEBALL##############################################################################################

In [290]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_curveball_movement)

# Implement KMeans
kmeans = KMeans(n_clusters=6)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_curveball_movement['Cluster'] = movement_labels

In [291]:
#summary of clusters
df_left_curveball_movement.groupby('Cluster').mean()

In [292]:
#value counts by cluster
df_left_curveball_movement['Cluster'].value_counts()

In [293]:
#rename clusters based on pitch description
df_left_curveball_movement['Cluster'] = df_left_curveball_movement['Cluster'].replace({0:'Slurve',1:'True Curve',2:'Small Curve',3:'True Curve',4:'Small Curve',5:'Slider Type'})

In [294]:
#change the name of cluster column to Pitch Movement
df_left_curveball_movement = df_left_curveball_movement.rename(columns={'Cluster':'Pitch Movement'})

In [295]:
#replace Player Name ID with Player Name from df_left_curveball
df_left_curveball_movement = df_left_curveball_movement.replace({'Player Name ID':df_left_curveball['Player Name']})

In [296]:
#export to csv
df_left_curveball_movement.to_csv('2022_left_curveball_movement.csv')

In [297]:
#import csv
df_left_curveball_movement = pd.read_csv('2022_left_curveball_movement.csv')

In [298]:
# Scale the data to normalize it
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_left_curveball_release)

# Implement KMeans
kmeans = KMeans(n_clusters=6)  # Number of clusters can be chosen depending on how you want to segregate
movement_labels = kmeans.fit_predict(scaled_data)


# Add the labels to your dataframe
df_left_curveball_release['Cluster'] = movement_labels

In [299]:
#summary of clusters
df_left_curveball_release.groupby('Cluster').mean()

In [300]:
#value counts by cluster
df_left_curveball_release['Cluster'].value_counts()

In [301]:
#rename clusters based on Release description
df_left_curveball_release['Cluster'] = df_left_curveball_release['Cluster'].replace({0:'Low 3/4 | Sidearm',1:'3/4',2:'3/4',3:'Low 3/4 | Sidearm',4:'3/4',5:'OverTheTop'})

In [302]:
#Change the name of cluster column to Pitch Release
df_left_curveball_release = df_left_curveball_release.rename(columns={'Cluster':'Pitch Release'})

In [303]:
#replace Player Name ID with Player Name from df_left_changeup
df_left_curveball_release = df_left_curveball_release.replace({'Player Name ID':df_left_curveball['Player Name']})

In [304]:
#export to csv
df_left_curveball_release.to_csv('2022_left_curveball_release.csv')

In [305]:
#import csv
df_left_curveball_release = pd.read_csv('2022_left_curveball_release.csv')

In [306]:
#combine movement and release dataframes for left handed curveballs based on Player Name ID
df_left_curveball_movement_release = pd.merge(df_left_curveball_movement, df_left_curveball_release, on='Player Name ID', how='outer')

In [307]:
#df_left_curveball_velo
#create 5 clusters based on velo percentile
#cluster 0 = 0-20 percentile
#cluster 1 = 20-40 percentile
#cluster 2 = 40-60 percentile
#cluster 3 = 60-80 percentile
#cluster 4 = 80-100 percentile
df_left_curveball_velo['Cluster'] = pd.cut(df_left_curveball_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/2791646568.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_curveball_velo['Cluster'] = pd.cut(df_left_curveball_velo['Velo_percentile'], bins=[0,.20,.40,.60,.80,1], labels=['0-20','20-40','40-60','60-80','80-100'])


In [308]:
#rename clusters based on velo percentile
df_left_curveball_velo['Cluster'] = df_left_curveball_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/3289844549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_curveball_velo['Cluster'] = df_left_curveball_velo['Cluster'].replace({'0-20':'Very Slow','20-40':'Slow','40-60':'Average','60-80':'Hard','80-100':'Very Hard'})


In [309]:
#rename cluster column to Pitch Velocity description
df_left_curveball_velo = df_left_curveball_velo.rename(columns={'Cluster':'Pitch Velocity'})

In [310]:
#replace Player Name ID with Player Name from df_left_curveball
df_left_curveball_velo = df_left_curveball_velo.replace({'Player Name ID':df_left_curveball['Player Name']})


In [311]:
#combine movement, release, and velo dataframes for left handed curveballs based on Player Name ID
df_left_curveball_movement_release_velo = pd.merge(df_left_curveball_movement_release, df_left_curveball_velo, on='Player Name ID', how='outer')

In [312]:
#df_left_curveball_movement_release_velo
#only include Player Name ID, Pitch Velocity, Pitch Movement, Pitch Release
df_left_curveball_movement_release_velo = df_left_curveball_movement_release_velo[['Player Name ID','Pitch Velocity','Pitch Movement','Pitch Release']]
# rename Player Name ID to Player Name in df_left_curveball_movement_release_velo
df_left_curveball_movement_release_velo = df_left_curveball_movement_release_velo.rename(columns={'Player Name ID':'Player Name'})

In [313]:
#df_left_curveball2
#df_left_curveball only include Player Name, Throws, Velocity, Vertical Break, Horizontal Break, Release Height, Side Release, Extension
df_left_curveball2 = df_left_curveball[['Player Name','Throws','Velocity','Vertical Break','Horizontal Break','Release Height','Side Release','Extension']]
#combine df_left_curveball2 and df_left_curveball_movement_release_velo if name appears in both dataframes
df_left_curveball_final = pd.merge(df_left_curveball2, df_left_curveball_movement_release_velo, on='Player Name', how='inner')

In [314]:
#delete duplicate rows if name appears in more than once in df_left_curveball_final
df_left_curveball_final = df_left_curveball_final.drop_duplicates(subset=['Player Name'])


In [315]:
#sort high to low by velocity
df_left_curveball_final = df_left_curveball_final.sort_values(by=['Velocity'], ascending=False)

In [316]:
df_left_curveball_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release
26,"Alvarado, José",Left,86.430769,-6.424615,-1.153846,6.505385,-0.699231,6.761538,Very Hard,Small Curve,3/4
966,"Phipps, Jackson",Left,84.937260,-10.228059,-4.749607,6.241154,0.251521,5.067039,Very Hard,Slider Type,3/4
1164,"Spencer, Jared",Left,84.517572,-4.056333,7.319540,5.873964,-2.257733,4.812148,Very Hard,Slider Type,3/4
264,"Cotto, Magdiel",Left,83.702268,-5.548493,3.553137,5.656515,-1.577901,5.336811,Very Hard,Small Curve,3/4
856,"Moore, Matt",Left,83.579245,-10.889057,4.055849,6.465388,-1.238532,6.224319,Very Hard,True Curve,OverTheTop
...,...,...,...,...,...,...,...,...,...,...,...
577,"Hock, Matthew",Left,63.199255,-11.190931,12.527195,5.148765,-0.788977,4.655288,Very Slow,True Curve,3/4
189,"Carlisle, Mason",Left,63.127890,-7.083180,12.449233,5.215893,-1.334417,5.053040,Very Slow,True Curve,3/4
528,"Hargrove, Cooper",Left,62.833179,-4.783124,11.637775,5.705077,-1.513962,4.425765,Very Slow,Slurve,3/4
106,"Betancourt, Anthony",Left,61.768481,-11.874264,12.144797,5.327197,-2.151700,4.237310,Very Slow,True Curve,3/4


In [317]:
# create a new column in df_left_fb_final called Pitch Type and set it equal to FB
df_left_fb_final['Pitch Type'] = 'FB'
df_left_slider_final['Pitch Type'] = 'SL'
df_left_changeup_final['Pitch Type'] = 'CH'
df_left_curveball_final['Pitch Type'] = 'CB'

In [318]:
#combine all 4 dataframes into one
df_left_final = pd.concat([df_left_fb_final, df_left_slider_final, df_left_changeup_final, df_left_curveball_final])

In [319]:
df_left_final

,Player Name,Throws,Velocity,Vertical Break,Horizontal Break,Release Height,Side Release,Extension,Pitch Velocity,Pitch Movement,Pitch Release,Pitch Type
1749,"Brown, Daniel",Left,98.881960,12.895436,-18.410347,6.338786,-1.284695,5.872700,Very Hard,Slight Sink,3/4,FB
14441,"Soto, Gregory",Left,98.666289,13.670142,-11.156941,5.507535,-2.182918,6.671955,Very Hard,Slight Ride,3/4,FB
251,"Alvarado, José",Left,98.075000,16.404000,-5.082000,6.540500,-0.751500,6.890000,Very Hard,Ride,OverTheTop,FB
6530,"Hader, Josh",Left,97.533007,17.700084,-8.553902,5.528112,-2.836951,6.776783,Very Hard,Ride,Low 3/4 | Sidearm,FB
2639,"Chapman, Aroldis",Left,97.389409,17.806108,-4.460099,6.070961,-0.690887,7.112808,Very Hard,Ride,OverTheTop,FB
...,...,...,...,...,...,...,...,...,...,...,...,...
577,"Hock, Matthew",Left,63.199255,-11.190931,12.527195,5.148765,-0.788977,4.655288,Very Slow,True Curve,3/4,CB
189,"Carlisle, Mason",Left,63.127890,-7.083180,12.449233,5.215893,-1.334417,5.053040,Very Slow,True Curve,3/4,CB
528,"Hargrove, Cooper",Left,62.833179,-4.783124,11.637775,5.705077,-1.513962,4.425765,Very Slow,Slurve,3/4,CB
106,"Betancourt, Anthony",Left,61.768481,-11.874264,12.144797,5.327197,-2.151700,4.237310,Very Slow,True Curve,3/4,CB


In [320]:
#new dataframe with description
df_left_final_description = df_left_final[['Player Name','Throws','Pitch Type','Pitch Velocity','Pitch Movement','Pitch Release']]

In [321]:
#combine columns Pitch Velocity','Pitch Movement','Pitch Release' into one column called Pitch Description
df_left_final_description['Pitch Description'] = df_left_final_description['Pitch Velocity'].astype(str) + ' ' + df_left_final_description['Pitch Movement'].astype(str) + ' ' + df_left_final_description['Pitch Release'].astype(str)

/var/folders/59/6bqdd_zs5_zdxml6x4n0v9mm0000gn/T/ipykernel_40142/1896389344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_left_final_description['Pitch Description'] = df_left_final_description['Pitch Velocity'].astype(str) + ' ' + df_left_final_description['Pitch Movement'].astype(str) + ' ' + df_left_final_description['Pitch Release'].astype(str)


In [322]:
#only include Player Name, Throws, Pitch Type, Pitch Description
df_left_final_description = df_left_final_description[['Player Name','Throws','Pitch Type','Pitch Description']]

In [323]:
#locate players
df_left_final_description.loc[df_left_final_description['Player Name'] == 'Frank, Hayden']

,Player Name,Throws,Pitch Type,Pitch Description
5431,"Frank, Hayden",Left,FB,Average Ride 3/4
377,"Frank, Hayden",Left,SL,Very Slow Gyro 3/4
446,"Frank, Hayden",Left,CH,Slow FB Shape OverTheTop
423,"Frank, Hayden",Left,CB,Average Slurve 3/4


In [324]:
#locate players
df_right_final_description.loc[df_right_final_description['Player Name'] == 'Brecht, Brody']

,Player Name,Throws,Pitch Type,Pitch Description
399,"Brecht, Brody",Right,SL,Very Hard Gyro 3/4
395,"Brecht, Brody",Right,CB,Very Hard True Curve 3/4
4924,"Brecht, Brody",Right,FB,Very Hard Cut-Ride OverTheTop


In [325]:
#combine right and left handed pitchers
df_final = pd.concat([df_right_final_description, df_left_final_description])

In [338]:
#look up player
df_final.loc[df_final['Player Name'] == 'Brecht, Brody']

,Player Name,Throws,Pitch Type,Pitch Description
399,"Brecht, Brody",Right,SL,Very Hard Gyro 3/4
395,"Brecht, Brody",Right,CB,Very Hard True Curve 3/4
4924,"Brecht, Brody",Right,FB,Very Hard Cut-Ride OverTheTop
